In [4]:
import pandas as pd 
import numpy as np

In [5]:
titanic = pd.read_csv('Downloads/train.csv')

In [6]:
titanic.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [ ]:
'''passengerid is not required 
    name is not required
    sex should be in binary form
    age should be continous by filling with average
    sibsp and parch should be combine together as both are making same sense
    ticket is not required
    cabin should be in binary form
    embarked is not required
'''

In [7]:
titanic['Cabin'] = np.where(titanic['Cabin'].isnull(),0,1)
titanic['Sex'] = titanic['Sex'].map({'male':0,'female':1})
titanic['family'] = titanic['SibSp']+titanic['Parch']
titanic['Age'].fillna(titanic['Age'].mean(),inplace=True)
titanic.drop(['PassengerId','Name','SibSp','Parch','Ticket','Embarked'],axis = 1,inplace=True)
titanic.head(10)

,Survived,Pclass,Sex,Age,Fare,Cabin,family
0,0,3,0,22.000000,7.2500,0,1
1,1,1,1,38.000000,71.2833,1,1
2,1,3,1,26.000000,7.9250,0,0
3,1,1,1,35.000000,53.1000,1,1
4,0,3,0,35.000000,8.0500,0,0
5,0,3,0,29.699118,8.4583,0,0
6,0,1,0,54.000000,51.8625,1,0
7,0,3,0,2.000000,21.0750,0,4
8,1,3,1,27.000000,11.1333,0,2
9,1,2,1,14.000000,30.0708,0,1


In [32]:
#write fully cleaned data
titanic.to_csv('cleaned.csv',index=False)         #index should not be written :first column

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
features = titanic.drop('Survived',axis=1)              #x = features,y= labels
labels = titanic['Survived']
x_train,x_test,y_train,y_test = train_test_split(features,labels,test_size = 0.4,random_state = 42)
x_test,x_val,y_test,y_val = train_test_split(x_test,y_test,test_size = 0.5, random_state = 42)
print(len(labels),len(x_train),len(x_test),len(x_val),len(y_train),len(y_test),len(y_val))
for size in [x_train,x_test,x_val]:
    print(str(round(len(size)/len(labels),2)*100)+'%')

891 534 178 179 534 178 179
60.0%
20.0%
20.0%


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import warnings 
warnings.filterwarnings('ignore',category=FutureWarning)

In [31]:
#five fold cross validation

rf = RandomForestClassifier()
cross_val_score(rf,x_train,y_train,cv=5)

array([0.86111111, 0.8317757 , 0.80373832, 0.80188679, 0.83018868])

In [32]:
#To find the optimal hyperparameter,we'll use grid search
from sklearn.model_selection import GridSearchCV

In [33]:
def print_result(results):
    print('BEST PARAMS {}'.format(results.best_params_))
    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean,std,params in zip(means,stds,results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean,2),round(std*2,2),params))

In [41]:
parameter={'n_estimators':[5,50,100],
           'max_depth':[2,10,20,None]}                  #how many random forest do you wanna build(n_estimators),
cv = GridSearchCV(rf,parameter,cv=5)                # 3(n_estimators)*4(max_depth)*5(cv) = 60 models
cv.fit(x_train,y_train.values.ravel())
print_result(cv)

BEST PARAMS {'max_depth': 10, 'n_estimators': 100}
0.76 (+/-0.15) for {'max_depth': 2, 'n_estimators': 5}
0.8 (+/-0.1) for {'max_depth': 2, 'n_estimators': 50}
0.79 (+/-0.13) for {'max_depth': 2, 'n_estimators': 100}
0.81 (+/-0.03) for {'max_depth': 10, 'n_estimators': 5}
0.82 (+/-0.05) for {'max_depth': 10, 'n_estimators': 50}
0.83 (+/-0.05) for {'max_depth': 10, 'n_estimators': 100}
0.8 (+/-0.07) for {'max_depth': 20, 'n_estimators': 5}
0.81 (+/-0.05) for {'max_depth': 20, 'n_estimators': 50}
0.81 (+/-0.03) for {'max_depth': 20, 'n_estimators': 100}
0.81 (+/-0.07) for {'max_depth': None, 'n_estimators': 5}
0.81 (+/-0.02) for {'max_depth': None, 'n_estimators': 50}
0.81 (+/-0.03) for {'max_depth': None, 'n_estimators': 100}


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [43]:
rf1 = RandomForestClassifier(n_estimators=100,max_depth=10)
rf1.fit(x_train,y_train.values.ravel())
rf2 = RandomForestClassifier(n_estimators=50,max_depth=10)
rf2.fit(x_train,y_train.values.ravel())
rf3 = RandomForestClassifier(n_estimators=50,max_depth=20)
rf3.fit(x_train,y_train.values.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [45]:
from sklearn.metrics import accuracy_score,precision_score,recall_score

In [47]:
for model in [rf1,rf2,rf3]:
    y_pred = model.predict(x_val)
    accuracy = round(accuracy_score(y_val,y_pred),2)
    precision = round(precision_score(y_val,y_pred),2)
    recall = round(recall_score(y_val,y_pred),2)
    print("max_depth : {} , n_estimators : {}, A:{} , P:{} , R:{} ".format(model.max_depth,model.n_estimators,accuracy,precision,recall))

max_depth : 10 , n_estimators : 100, A:0.84 , P:0.86 , R:0.74 
max_depth : 10 , n_estimators : 50, A:0.82 , P:0.84 , R:0.7 
max_depth : 20 , n_estimators : 50, A:0.83 , P:0.84 , R:0.75 


In [51]:
y_pred = rf1.predict(x_test)
accuracy = round(accuracy_score(y_test,y_pred),2)
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
print("max_depth : {} , n_estimators : {}, A:{} , P:{} , R:{} ".format(rf1.max_depth,
                                                                           rf1.n_estimators,accuracy,precision,recall))

max_depth : 10 , n_estimators : 100, A:0.8 , P:0.76 , R:0.65 
